<a href="https://colab.research.google.com/github/GHunor/CMO_NN/blob/main/CMO_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [20]:
from zipfile import ZipFile
import os

from pathlib import Path

In [21]:
labels_path = "/content/dataset_all.zip"
zip_file = ZipFile(labels_path)
zip_file.extractall("./DS_o")

labels_path = "/content/DS_o/dataset_labels.zip"
#zip_path = keras.utils.get_file(origin=uri, fname="data_labels0.zip")
zip_file = ZipFile(labels_path)
zip_file.extractall("./labels")

labels_path = "/content/DS_o/dataset_myo.zip"
zip_file = ZipFile(labels_path)
zip_file.extractall("./myo")

labels_path = "/content/DS_o/dataset_opt.zip"
zip_file = ZipFile(labels_path)
zip_file.extractall("./op")

In [22]:
L_ADRS = [];
hp = './labels'
p = '/'
for child in Path(hp).iterdir():
    if child.is_file():
        L_ADRS.append(child.name)
labels = [];
for adrs in L_ADRS:
    df = np.genfromtxt(hp + p + adrs, delimiter=",")#df = pd.read_csv(hp + p + adrs, header=None)
    labels.append(df)

In [23]:
M_ADRS = [];
hp = './myo'
p = '/'
for child in Path(hp).iterdir():
    if child.is_file():
        M_ADRS.append(child.name)
myo = [];
for adrs in M_ADRS:
    df = np.genfromtxt(hp + p + adrs, delimiter=",")#= pd.read_csv(hp + p + adrs, header=None)
    myo.append(df)

In [24]:
O_ADRS = [];
hp = './op'
p = '/'
for child in Path(hp).iterdir():
    if child.is_file():
        O_ADRS.append(child.name)
op = [];
for adrs in O_ADRS:
    df = np.genfromtxt(hp + p + adrs, delimiter=",")#df = pd.read_csv(hp + p + adrs, header=None)
    op.append(df)

In [25]:
"""
  df = pd.read_csv('./labels/' + ADRS[0])
  labels = [];
  labels.append(df)
  df = pd.read_csv('./labels/' + ADRS[1])
  labels.append(df)
  print(labels)
  print(labels[1])
  print(df)
"""

"\n  df = pd.read_csv('./labels/' + ADRS[0])\n  labels = [];\n  labels.append(df)\n  df = pd.read_csv('./labels/' + ADRS[1])\n  labels.append(df)\n  print(labels)\n  print(labels[1])\n  print(df)\n"

In [26]:
#ADRS = pd.read_csv(csv_path)
#df = pd.read_csv(ADRS(i))


In [27]:
# create dataset and labels

#model.reset_states() before feeding the new dataset and calling model.fit()
# only myo 15 moves

# opto too 15 moves

# Have a CNN run thorugh 10 datappoint (one data point represents 10 ns)
# 1. only CNN (17->18-20)
# 2. only LSTM
# 3. Both

In [28]:
print(np.size(myo))

65


/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).size


In [29]:
x_val = myo[0]
y_val = labels[0]

sequence_length = 20
step = 1
batch_size = 64
learning_rate = 0.001
epochs = 5

dataset_val = keras.preprocessing.timeseries_dataset_from_array(
    x_val,
    y_val,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
)

for batch in dataset_val.take(1):
    inputs, targets = batch

In [30]:
"""
i = 1;
len = np.size(labels[i], 0)
print(len)
print(labels[0])
"""

'\ni = 1;\nlen = np.size(labels[i], 0)\nprint(len)\nprint(labels[0])\n'

In [31]:
# will be repeated
DS_T = [];
for i, M in enumerate(myo[1:]):
  if np.size(M, 0)<sequence_length:
    continue
  i = i+1
  x_train = M
  ishape = []
  ishape.append(1)
  ishape.append(np.size(M, 1))
  y_train = labels[i]
  oshape = []
  oshape.append(1)
  oshape.append(np.size(labels[i], 1))
  dataset_train = keras.preprocessing.timeseries_dataset_from_array(
      x_train,
      y_train,
      sequence_length=sequence_length,
      sampling_rate=step,
      batch_size=batch_size,
  )
  DS_T.append(dataset_train)


In [32]:
# will be repeated



In [33]:

path_checkpoint1 =  "model1_checkpoint.h5"
path_checkpoint2 =  "model2_checkpoint.h5"
path_checkpoint3 =  "model3_checkpoint.h5"

es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

modelckpt_callback1 = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint1,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)
modelckpt_callback2 = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint2,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)
modelckpt_callback3 = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint3,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)

In [34]:
#models
#input_shape = (batch_size, sequence_length, ishape[1])
inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
#inputs = keras.layers.Input(shape=(ishape[0], ishape[1]))
#inputs = keras.layers.Input(shape=(1,17))


lstm_out = keras.layers.LSTM(32)(inputs)
LSoutputs = keras.layers.Dense(oshape[1])(lstm_out)

"""
cnn_out1 = keras.layers.Conv1D(32, 1, activation='relu',input_shape=(inputs.shape[1], inputs.shape[2]))(inputs)
CNNoutputs = keras.layers.Dense(oshape[1])(cnn_out1)
model1 = keras.Model(inputs=inputs, outputs=CNNoutputs)
model1.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
model1.summary()
"""

#x = tf.ones((batch_size, inputs.shape[1], inputs.shape[1]))
#y = model1(x)
#model1.summary()

model2 = keras.Model(inputs=inputs, outputs=LSoutputs)
model2.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
model2.summary()

"""
lstm_out2 = keras.layers.LSTM(32)(cnn_out1)
COMBoutputs = keras.layers.Dense(oshape[1])(lstm_out2)
model3 = keras.Model(inputs=inputs, outputs=COMBoutputs)
model3.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
model3.summary()
"""

# model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
# model.summary()


modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint1,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20, 8)]           0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                5248      
                                                                 
 dense_1 (Dense)             (None, 15)                495       
                                                                 
Total params: 5,743
Trainable params: 5,743
Non-trainable params: 0
_________________________________________________________________


In [35]:
# will be repeated
for dataset_train in DS_T:
  print(dataset_train)
  history2 = model2.fit(
      dataset_train,
      epochs=epochs,
      validation_data=dataset_val,
      callbacks=[es_callback, modelckpt_callback2],
  )
  model2.reset_states() 

<BatchDataset element_spec=(TensorSpec(shape=(None, None, 8), dtype=tf.float64, name=None), TensorSpec(shape=(None, 15), dtype=tf.float64, name=None))>
Epoch 1/5
1/1 [==============================] - ETA: 0s - loss: 0.5237
Epoch 1: val_loss improved from inf to 0.47504, saving model to model2_checkpoint.h5
1/1 [==============================] - 2s 2s/step - loss: 0.5237 - val_loss: 0.4750
Epoch 2/5
1/1 [==============================] - ETA: 0s - loss: 0.4968
Epoch 2: val_loss improved from 0.47504 to 0.45394, saving model to model2_checkpoint.h5
1/1 [==============================] - 0s 63ms/step - loss: 0.4968 - val_loss: 0.4539
Epoch 3/5
1/1 [==============================] - ETA: 0s - loss: 0.4717
Epoch 3: val_loss improved from 0.45394 to 0.43424, saving model to model2_checkpoint.h5
1/1 [==============================] - 0s 65ms/step - loss: 0.4717 - val_loss: 0.4342
Epoch 4/5
1/1 [==============================] - ETA: 0s - loss: 0.4480
Epoch 4: val_loss improved from 0.43424 t

In [36]:
score = model.evaluate(x_test, y_test, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

NameError: ignored

In [37]:
#preprocess time series into "pics"
len = np.size(myo[0], 0)
h = 8
TrainDx = np.empty([0, h, 8, 1]).astype('float32')
TrainDy = []
TestDx = []
TestDy = []
for j in range(len-h):
  #TD.append(myo[i:(h+1), 1:3])
  TestDx.append(myo[0][j:j+h])
  TestDy.append(labels[0][j+h-1:j+h])

for k, L in  enumerate(myo[1:]):
  print("k:")
  print(k)
  if np.size(L, 0)<h:
    continue
  len2 = np.size(L, 0)
  for j in range(len2-h):
    print("j:")
    print(j)
    #TD.append(myo[i:(h+1), 1:3])
    arr =  np.asarray(L[j:j+h])
    arr = np.reshape(arr, [1,  h, 8, 1])
    print(arr.shape)
    print(TrainDx.shape)
    TrainDx = np.append(TrainDx, arr, axis=0)
    TrainDy.append(np.asarray(labels[k+1][j+h-1:j+h]))
print(np.size(TrainDx))
print("TrainDx[0]:")
print(TrainDx[0])
print("TrainDx size:")
print(np.size(TrainDx))
print(np.size(TrainDx, 0))
print(np.size(TrainDx[0], 0))
#print(np.size(TrainDx, 2))
print(TrainDy[0])
print(np.size(TrainDy, 0))
#print(np.size(TrainDy, 1))
#print(labels[0])


Streaming output truncated to the last 5000 lines.
(799, 8, 8, 1)
j:
6
(1, 8, 8, 1)
(800, 8, 8, 1)
j:
7
(1, 8, 8, 1)
(801, 8, 8, 1)
j:
8
(1, 8, 8, 1)
(802, 8, 8, 1)
j:
9
(1, 8, 8, 1)
(803, 8, 8, 1)
j:
10
(1, 8, 8, 1)
(804, 8, 8, 1)
j:
11
(1, 8, 8, 1)
(805, 8, 8, 1)
j:
12
(1, 8, 8, 1)
(806, 8, 8, 1)
j:
13
(1, 8, 8, 1)
(807, 8, 8, 1)
j:
14
(1, 8, 8, 1)
(808, 8, 8, 1)
j:
15
(1, 8, 8, 1)
(809, 8, 8, 1)
j:
16
(1, 8, 8, 1)
(810, 8, 8, 1)
j:
17
(1, 8, 8, 1)
(811, 8, 8, 1)
j:
18
(1, 8, 8, 1)
(812, 8, 8, 1)
j:
19
(1, 8, 8, 1)
(813, 8, 8, 1)
j:
20
(1, 8, 8, 1)
(814, 8, 8, 1)
j:
21
(1, 8, 8, 1)
(815, 8, 8, 1)
j:
22
(1, 8, 8, 1)
(816, 8, 8, 1)
j:
23
(1, 8, 8, 1)
(817, 8, 8, 1)
j:
24
(1, 8, 8, 1)
(818, 8, 8, 1)
j:
25
(1, 8, 8, 1)
(819, 8, 8, 1)
j:
26
(1, 8, 8, 1)
(820, 8, 8, 1)
j:
27
(1, 8, 8, 1)
(821, 8, 8, 1)
j:
28
(1, 8, 8, 1)
(822, 8, 8, 1)
j:
29
(1, 8, 8, 1)
(823, 8, 8, 1)
j:
30
(1, 8, 8, 1)
(824, 8, 8, 1)
k:
27
j:
0
(1, 8, 8, 1)
(825, 8, 8, 1)
j:
1
(1, 8, 8, 1)
(826, 8, 8, 1)
j:
2
(1, 8, 8, 1

In [ ]:
"""
print(np.size(TrainDx))
print(np.size(TrainDx, 0))
TrainDx = np.asarray(TrainDx)
print(np.size(TrainDx))
print(np.size(TrainDx, 0))
arr = np.array(TrainDx)
print(np.size(arr))
print(np.size(arr, 0))
TrainDx = TrainDx.reshape(np.size(TrainDx, 0), h, 8, 1)
"""

In [38]:
print(TrainDx.dtypes)
print(TrainDy.dtypes)

AttributeError: ignored

In [39]:

TrainDy = np.asarray(TrainDy).astype('float32')

<ipython-input-39-d7ba68a995a8>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  TrainDy = np.asarray(TrainDy).astype('float32')


ValueError: ignored

In [ ]:
TestDx = np.asarray(TestDx).astype('float32')
TestDx = TestDx.reshape(np.size(TestDx, 0), np.size(TestDx, 1), np.size(TestDx, 2), 1)

TestDy = np.asarray(TestDy).astype('float32')

In [ ]:
input_shape = TrainDx.shape
print(input_shape)
Kern1 = [2,1,1]
Kern2 = [3,3,2]
Kern = Kern2

In [ ]:

print(input_shape)
inputs = keras.layers.Input(shape=input_shape[1:])
cnn_out1 = keras.layers.Conv2D(16, kernel_size = (3, Kern[0]), activation = 'relu', input_shape = input_shape[1:])(inputs)
cnn_out2 = keras.layers.Conv2D(32, kernel_size = (3, Kern[1]), activation = 'relu')(cnn_out1)
cnn_out3 = keras.layers.MaxPooling2D(pool_size = (2, Kern[2]))(cnn_out2)
cnn_out4 = keras.layers.Flatten()(cnn_out3)
cnn_out5 = keras.layers.Dense(64, activation = 'relu')(cnn_out4)
CNNoutputs = keras.layers.Dense(oshape[1], activation = 'softmax')(cnn_out5)
model1 = keras.Model(inputs=inputs, outputs=CNNoutputs)
model1.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
model1.summary()

history1 = model1.fit(
  TrainDx, TrainDy,
  batch_size = batch_size,
  epochs = epochs,
  verbose = 1,
  validation_data = (TestDx, TestDy),
  callbacks=[es_callback, modelckpt_callback1]
)

In [ ]:
MO = []

for i, L in enumerate(myo):
  #print(L)
  MOo = []
  for j in range(np.size(L, 0)):
    Lo = L[j:j+1]
    Lo = Lo.values.tolist()
    Lo = Lo[0]
    opL = op[i].values.tolist()
    P = opL[j:j+1]
    P = P[0]
    for t in k:
      Lo.append(t)
    MOo.append(Lo)
  MO.append(MOo)
print(np.size(MO, 0))
print(np.size(MO, 1))
print(np.size(MO, 2))

In [ ]:
# concat myo with opt
x_val = MO[0]
x_val = np.asarray(x_val)
print(np.size(x_val, 0))
print(np.size(x_val, 1))
y_val = labels[0]
y_val = np.asarray(y_val)
print(np.size(y_val, 0))
print(np.size(y_val, 1))

sequence_length = 40
step = 1
batch_size = 64
learning_rate = 0.001
epochs = 5

dataset_val = keras.preprocessing.timeseries_dataset_from_array(
    x_val,
    y_val,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
)

for batch in dataset_val.take(1):
    inputs, targets = batch
print(inputs.shape[0])
print(inputs.shape[1])

DS_T = [];
for i, M in enumerate(MO[1:]):
  if np.size(M, 0)<sequence_length:
    continue
  i = i+1
  x_train = M
  ishape = []
  ishape.append(1)
  ishape.append(np.size(M, 1))
  y_train = labels[i]
  oshape = []
  oshape.append(1)
  oshape.append(np.size(labels[i], 1))
  dataset_train = keras.preprocessing.timeseries_dataset_from_array(
      x_train,
      y_train,
      sequence_length=sequence_length,
      sampling_rate=step,
      batch_size=batch_size,
  )
  DS_T.append(dataset_train)

path_checkpoint3 =  "model3_checkpoint.h5"
path_checkpoint4 =  "model4_checkpoint.h5"
es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

modelckpt_callback3 = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint3,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)
modelckpt_callback4 = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint4,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)


inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
lstm_out = keras.layers.LSTM(32)(inputs)
LSoutputs = keras.layers.Dense(oshape[1])(lstm_out)


model4 = keras.Model(inputs=inputs, outputs=LSoutputs)
model4.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
model4.summary()

for dataset_train in DS_T:
  print(dataset_train)
  history4 = model4.fit(
      dataset_train,
      epochs=epochs,
      validation_data=dataset_val,
      callbacks=[es_callback, modelckpt_callback4],
  )
  model2.reset_states() 

len = np.size(MO[0], 0)
h = 10
TrainDx = []
TrainDy = []
TestDx = []
TestDy = []
for j in range(len-h):
  #TD.append(myo[i:(h+1), 1:3])
  TestDx.append(MO[0][j:j+h])
  TestDy.append(labels[0][j+h-1:j+h])

for k, L in  enumerate(MO[1:]):
  if np.size(L, 0)<h:
    continue
  for j in range(len-h):
    #TD.append(myo[i:(h+1), 1:3])
    TrainDx.append(L[j:j+h])
    TrainDy.append(labels[k+1][j+h-1:j+h])

TrainDx = np.asarray(TrainDx)
TrainDx = TrainDx.reshape(np.size(TrainDx, 0), np.size(TrainDx, 1), np.size(TrainDx, 2), 1)

TrainDy = np.asarray(TrainDy)

TestDx = np.asarray(TestDx)
TestDx = TestDx.reshape(np.size(TestDx, 0), np.size(TestDx, 1), np.size(TestDx, 2), 1)

TestDy = np.asarray(TestDy)

input_shape = (np.size(TrainDx, 0), np.size(TrainDx, 1), np.size(TrainDx, 2), 1)
Kern1 = [2,1,1]
Kern2 = [3,3,2]
Kern = Kern1

inputs = keras.layers.Input(shape=input_shape[1:])
cnn_out1 = keras.layers.Conv2D(16, kernel_size = (3, Kern[0]), activation = 'relu', input_shape = input_shape[1:])(inputs)
cnn_out2 = keras.layers.Conv2D(32, kernel_size = (3, Kern[1]), activation = 'relu')(cnn_out1)
cnn_out3 = keras.layers.MaxPooling2D(pool_size = (2, Kern[2]))(cnn_out2)
cnn_out4 = keras.layers.Flatten()(cnn_out3)
cnn_out5 = keras.layers.Dense(64, activation = 'relu')(cnn_out4)
CNNoutputs = keras.layers.Dense(oshape[1], activation = 'softmax')(cnn_out5)
model3 = keras.Model(inputs=inputs, outputs=CNNoutputs)
model3.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
model3.summary()

history3 = model3.fit(
  TrainDx, TrainDy,
  batch_size = batch_size,
  epochs = epochs,
  verbose = 1,
  validation_data = (TestDx, TestDy),
  callbacks=[es_callback, modelckpt_callback3]
)

In [ ]:
"""
import keras
from keras.datasets import mnist
from keras.sampleEducbaModels import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as sampleEducba
import numpy as np
#Loading the mnist dataset for the sample purpose
(TrainV1, TrainV2), (TestV1, TestV2) = mnist.load_data()
#modify the existing data set for our requirement of the sampleEducbaModel.
rImg, columnsOfImage = 28, 28
if sampleEducba.image_data_format() == 'channels_first':
  TrainV1 = TrainV1.reshape(TrainV1.shape[0], 1, rImg, columnsOfImage)
  TestV1 = TestV1.reshape(TestV1.shape[0], 1, rImg, columnsOfImage)
  input_shape = (1, rImg, columnsOfImage)
else:
  TrainV1 = TrainV1.reshape(TrainV1.shape[0], rImg, columnsOfImage, 1)
  TestV1 = TestV1.reshape(TestV1.shape[0], rImg, columnsOfImage, 1)
  input_shape = (rImg, columnsOfImage, 1)
TrainV1 = TrainV1.astype('float32')
TestV1 = TestV1.astype('float32')
TrainV1 /= 255
TestV1 /= 255
TrainV2 = keras.utils.to_categorical(TrainV2, 10)
TestV2 = keras.utils.to_categorical(TestV2, 10)
#Model creation
Model = Sequential()
Model.add(Conv2D(32, kernel_size = (3, 3),
activation = 'relu', input_shape = input_shape))
Model.add(Conv2D(64, (3, 3), activation = 'relu'))
Model.add(MaxPooling2D(pool_size = (2, 2)))
Model.add(Dropout(0.25)) 
Model.add(Flatten())
Model.add(Dense(128, activation = 'relu'))
Model.add(Dropout(0.5))
Model.add(Dense(10, activation = 'softmax'))
# Model compilation
Model.compile(loss = keras.losses.categorical_crossentropy,
optimizer = keras.optimizers.Adadelta(), metrics = ['accuracy'])
# training the sampleEducbaModel by using fit function
Model.fit(
  TrainV1, TrainV2,
  batch_size = 128,
  epochs = 12,
  verbose = 1,
  validation_data = (TestV1, TestV2)
)